<a href="https://colab.research.google.com/github/humanprodev/azure-search-openai-demo/blob/main/STEP_1_aistudio_gemini_prompt_freeform_b64.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

In [2]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import io
import json
import mimetypes
import pathlib
import pprint
import requests

import PIL.Image
import IPython.display
from IPython.display import Markdown

### Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [4]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [5]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [6]:
model = 'gemini-pro' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MC41LCJ0b3BfcCI6MSwidG9wX2siOjEsIm1heF9vdXRwdXRfdG9rZW5zIjoyMDQ4LCJzdG9wX3NlcXVlbmNlcyI6W119' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [7]:
contents

[{'parts': [{'text': 'Change discussion notes about certain competency of the person into well written, engaging assessment report that describes said competency of the person. When possible, use verb and actions to describe predicted behavior instead of judgmental traits. Create narrative that predicts behavior, instead of merely repeating the points.'},
   {'text': "input: DRIVE AND COURAGE\nScore: 3/5\nRequirement: 3/5\n* mau menjalankan dengan total, \n* ada unsur masih 'menjaga drinya' berusaha mengkalkulasi yang dikerjain, walau emang masih nahan dan mikir biar diterima\n* belum ngebakar2 orang2 disekitarnya\n* merubah jam kerja -- berani melakukan perubahan yang memang risiko ya\n* lebih masih mengajak untuk bekerja dengan baik, menanamkan sikap kerja yang baik. belum antusiaske"},
   {'text': 'output: DRIVE AND COURAGE\nScore: 3/5\nRequirement: 3/5\nMemiliki drive yang cukup kuat, dan terlihat antusias dalam bekerja. Menjalankan tugas dan mengupayakan penanganan pekerjaan denga

In [8]:
generation_config

{'temperature': 0.5,
 'top_p': 1,
 'top_k': 1,
 'max_output_tokens': 2048,
 'stop_sequences': []}

In [9]:
safety_settings

[{'category': 'HARM_CATEGORY_HARASSMENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_HATE_SPEECH',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'}]

### Load image data from Drive-IDs

In [10]:
for content in contents:
  for n, part in enumerate(content['parts']):
    if image:=part.get('image', None):
      if drive_id:=image.get('drive_id', None):
        path = next(pathlib.Path(f'/gdrive/.shortcut-targets-by-id/{drive_id}').glob('*'))
        data = path.read_bytes()
        mime_type, _ = mimetypes.guess_type(path)
      elif image_url:=image.get('image_url', None):
        response = requests.get(image_url)
        data = response.content
        mime_type = response.headers['content-type']
      else:
        raise ValueError('Either drive_id or image_url must be provided.')

      if mime_type is None:
        # Guess!
        mime_type = 'image/png'

      blob = {'data': data, 'mime_type': mime_type}
      content['parts'][n] = {'inline_data': blob}

In [11]:
import IPython.display
import PIL.Image
import io

for content in contents:
    for part in content['parts']:
        if text := part.get('text', None):
            print(text)
        elif data := part.get('inline_data', None):
            img = PIL.Image.open(io.BytesIO(data['data']))
            img.thumbnail([512,512])
            IPython.display.display(img)
    print('_'*80)


Change discussion notes about certain competency of the person into well written, engaging assessment report that describes said competency of the person. When possible, use verb and actions to describe predicted behavior instead of judgmental traits. Create narrative that predicts behavior, instead of merely repeating the points.
input: DRIVE AND COURAGE
Score: 3/5
Requirement: 3/5
* mau menjalankan dengan total, 
* ada unsur masih 'menjaga drinya' berusaha mengkalkulasi yang dikerjain, walau emang masih nahan dan mikir biar diterima
* belum ngebakar2 orang2 disekitarnya
* merubah jam kerja -- berani melakukan perubahan yang memang risiko ya
* lebih masih mengajak untuk bekerja dengan baik, menanamkan sikap kerja yang baik. belum antusiaske
output: DRIVE AND COURAGE
Score: 3/5
Requirement: 3/5
Memiliki drive yang cukup kuat, dan terlihat antusias dalam bekerja. Menjalankan tugas dan mengupayakan penanganan pekerjaan dengan sungguh-sungguh, bertanggung jawab untuk memberikan deliverabl

### Call the API

In [12]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False)

In [13]:
if generation_config.get('candidate_count', 1) == 1:
  display(Markdown(response.text))
else:
  print(response.candidates)

COLLABORATION
Score: 3/5
Requirement: 3/5
Sikap interpersonalnya yang luwes membantu dia untuk berinteraksi dengan orang-orang di dalam tugasnya. Ada empati, merasa tujuan kelompok menjadi hal yang penting dan utama. Konsisten --> hal ini berkembang jauh apabila dibandingkan dengan PCP yang lalu, karena pengalaman dan referensi yang banyak membuat dirinya semakin yakin.

In [14]:
response.candidates

[content {
  parts {
    text: "COLLABORATION\nScore: 3/5\nRequirement: 3/5\nSikap interpersonalnya yang luwes membantu dia untuk berinteraksi dengan orang-orang di dalam tugasnya. Ada empati, merasa tujuan kelompok menjadi hal yang penting dan utama. Konsisten --> hal ini berkembang jauh apabila dibandingkan dengan PCP yang lalu, karena pengalaman dan referensi yang banyak membuat dirinya semakin yakin."
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [15]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}